# Huggingface Tokenizer

## 학습 목표
1. Huggingface tokenizer 사용법을 살펴본다.
2. 한글, 영어, multilingual tokenizer를 비교한다.

**Context**
1. Huggingface tokenizer 소개
2. BERT, KLUE BERT, mBERT tokenizer 비교

In [ ]:
!pip install transformers

## 1. Huggingface tokenizer 소개



```
💡 Huggingface 란❓

자연어처리(NLP) 태스크를 수행하기 위한 사전학습된 언어 모델(PLM, pre-trained language model) 및 학습에 유용한 다양한 유틸리티들을 모아둔 라이브러리다.

Transformer 기반 언어 모델(e.g., BERT, GPT, BART) 등 다양한 모델의 pre-trained weight를 바로 불러올 수 있다.
각 언어 모델과 같이 학습된 tokenizer 또한 불러올 수 있는데, 이는 사용자가 직접 tokenizer를 학습할 수고를 덜어준다.

Tokenizer를 학습한다는 것은, character-based나 word-based tokenizer 보다 subword-based tokenizer가 해당이 되는데,
모델이 본 토큰이 많으면 많을수록 좀 더 정확하게 쪼갤 수 있다.
예를 들어, 단어 "bluetooth"를 토큰화 한다면 "blue", "tooth"로 쪼개질 수 있지만,
이럴 경우 모델이 '블루투스' 라는 의미보다 '파란 치아'라는 의미로 이해할 것이다.
반면 단어 블루투스를 많이 보고 학습된 모델은 "bluetooth"를 두 단어로 쪼개지 않을 것이다.

이렇게 사전에 방대한 데이터로 학습된 PLM 및 tokenizer를 사용한다면, 자연어처리 태스크에서 좀 더 높은 성능을 기대할 수 있다.
```

```
💡 Huggingface에는 많은 language model이 존재하는 만큼 그에 해당하는 많은 tokenizer이 존재한다.

모델 이름을 따라 BERTTokenizer, GPT2Tokenizer 등 다양하게 불러올 수 있지만,
모델의 종류가 점점 많아지고 있기 때문에 이런 클래스의 이름들을 일일이 다 기억하기 어렵다.
모든 클래스들을 하나로 모아둔 클래스가 있는데 이것을 auto class라고 부른다.

Huggingface에서는 auto class를 제공하는데 이는 pre-trained model path에 따라 모델을 유추한다.
예를 들어 BERT와 GPT의 tokenizer를 불러오기 위해서는 BERTTokenizer과 GPT2Tokenizer를 각각 import 해주어야 하지만,
auto class를 사용한다면 AutoTokenizer 클래스 하나만 import 해주어도 된다.
```

In [ ]:
import torch
from transformers import AutoTokenizer

> 💡 사전 학습된 tokenizer나 model을 불러올 때는 `from_pretrained(path)`를 사용한다. 

In [ ]:
bert_tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

> Tokenizer를 프린트하면 tokenizer의 정보를 확인할 수 있다.

In [ ]:
print(bert_tokenizer)

BertTokenizerFast(name_or_path='bert-base-uncased', vocab_size=30522, model_max_length=512, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'})


> 다음 string으로 BERT tokenizer의 다양한 함수들을 살펴보도록 한다.

In [ ]:
example_str = "This is a tutorial for huggingface tokenizers."

>`tokenize(string)`
- 주어진 string을 tokenizer의 vocab을 사용하여 tokenizing 한다.

In [ ]:
tokens = bert_tokenizer.tokenize(example_str)
print(tokens)

['this', 'is', 'a', 'tutor', '##ial', 'for', 'hugging', '##face', 'token', '##izer', '##s', '.']


>`convert_tokens_to_ids(tokens)`
- 주어진 token or list of tokens를 모델 임베딩 인덱스로 변환한다.

In [ ]:
input_ids = bert_tokenizer.convert_tokens_to_ids(tokens)
print(input_ids)

[2023, 2003, 1037, 14924, 4818, 2005, 17662, 12172, 19204, 17629, 2015, 1012]


>`encode`
- 위에 두 함수를 합친 함수다 `convert_tokens_to_ids(tokenize(text))`
- BERT tokenizer의 경우 앞뒤로 CLS token id(101)와 SEP token id(102)가 붙는다.


In [ ]:
input_ids2 = bert_tokenizer.encode(example_str)
print(input_ids2)

[101, 2023, 2003, 1037, 14924, 4818, 2005, 17662, 12172, 19204, 17629, 2015, 1012, 102]


>`__call__`
- 토크나이저를 직접 부르면 인코딩 뿐만 아니라 해당 모델에 들어가야할 입력들도 자동으로 만들어준다.
- 예를 들면 BERT의 경우 `input_ids`, `attention_mask`, `token_type_ids`가 dictionary 형태를 리턴한다.
- 이전에는 `encode_plus` 및 `batch_encode_plus` 함수를 사용했지만 이 함수들은 더이상 사용하지 않고 tokenizer를 직접 부르는 방식으로 변경되었다.

In [ ]:
input_ids_plus = bert_tokenizer(example_str)
print(input_ids_plus)

{'input_ids': [101, 2023, 2003, 1037, 14924, 4818, 2005, 17662, 12172, 19204, 17629, 2015, 1012, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}


> 배치 내의 문장들을 한꺼번에 인코딩 할 수도 있다.

In [ ]:
batch_example = [
    "This is a simple batch example.",
    "Can it decode multiple sentences at the same time?",
    "You can use the batch_decode function to do it.",
    "Let's encode these sentences first, then perform batch_decode."
]

input_ids_batch = bert_tokenizer(batch_example)
print(input_ids_batch)

{'input_ids': [[101, 2023, 2003, 1037, 3722, 14108, 2742, 1012, 102], [101, 2064, 2009, 21933, 3207, 3674, 11746, 2012, 1996, 2168, 2051, 1029, 102], [101, 2017, 2064, 2224, 1996, 14108, 1035, 21933, 3207, 3853, 2000, 2079, 2009, 1012, 102], [101, 2292, 1005, 1055, 4372, 16044, 2122, 11746, 2034, 1010, 2059, 4685, 14108, 1035, 21933, 3207, 1012, 102]], 'token_type_ids': [[0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]}


> `padding=True` 파라미터를 주면 배치 내의 가장 긴 텍스트 길이 기준으로 자동으로 패딩을 해준다.

In [ ]:
bert_tokenizer(batch_example, padding=True)

{'input_ids': [[101, 2023, 2003, 1037, 3722, 14108, 2742, 1012, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0], [101, 2064, 2009, 21933, 3207, 3674, 11746, 2012, 1996, 2168, 2051, 1029, 102, 0, 0, 0, 0, 0], [101, 2017, 2064, 2224, 1996, 14108, 1035, 21933, 3207, 3853, 2000, 2079, 2009, 1012, 102, 0, 0, 0], [101, 2292, 1005, 1055, 4372, 16044, 2122, 11746, 2034, 1010, 2059, 4685, 14108, 1035, 21933, 3207, 1012, 102]], 'token_type_ids': [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]}

> `truncation=True` 파라미터를 주면 모델의 `max_seq_length`나 사용자가 정의한 길이로 입력 문장들을 잘라준다. BERT의 경우 `max_seq_length`는 512다.

In [ ]:
super_long_example = "Lorem ipsum dolor sit amet, consectetur adipiscing elit. Vestibulum sed nulla ultrices, ornare quam id, condimentum ante. Pellentesque eu mattis enim. Fusce eu porta lacus, sed porttitor neque. Ut vitae diam quis velit sollicitudin suscipit. Curabitur efficitur metus et risus rutrum, vel elementum magna egestas. Etiam quis elementum lectus. Vivamus non erat molestie, ornare eros eu, ornare nisi. Duis ullamcorper ex convallis, ullamcorper nisi quis, auctor nisi. Aenean sapien felis, pellentesque ut purus eu, finibus pretium tellus. Aliquam elementum faucibus libero luctus placerat. Suspendisse turpis neque, malesuada ac commodo eget, rhoncus nec lacus. Suspendisse et lectus ante." + \
                     "Vivamus faucibus dui nisl, sit amet pellentesque justo rutrum eleifend. In at ipsum varius, pellentesque massa nec, luctus mauris. Mauris porttitor mi urna, at faucibus diam fermentum et. Fusce ac egestas neque. Quisque sed rutrum elit. Morbi rhoncus feugiat est a fringilla. Nulla nunc erat, volutpat aliquet condimentum in, sollicitudin non nisi. Suspendisse rutrum eros ut enim suscipit, at aliquet ipsum porttitor. Vivamus efficitur, ante id pretium suscipit, risus orci tempus dui, eget pellentesque massa augue tincidunt neque. Curabitur ultricies eget erat eget scelerisque. Proin et fermentum enim, quis condimentum nisi. Nunc eleifend diam non diam blandit, vitae iaculis urna pharetra. Mauris nec nunc volutpat, eleifend turpis condimentum, imperdiet nulla. Nunc augue nulla, laoreet et dui at, porttitor tempus ligula. Phasellus quis mi ac dolor tincidunt congue."
super_long_id = bert_tokenizer(super_long_example).input_ids
truncated_super_long_id = bert_tokenizer(super_long_example, truncation=True).input_ids

print(super_long_id)
print(len(super_long_id))
print(len(truncated_super_long_id))

Token indices sequence length is longer than the specified maximum sequence length for this model (575 > 512). Running this sequence through the model will result in indexing errors


[101, 19544, 2213, 12997, 17421, 2079, 10626, 4133, 2572, 3388, 1010, 9530, 3366, 6593, 3388, 3126, 27133, 18136, 6129, 12005, 2102, 1012, 17447, 12322, 25100, 7367, 2094, 19701, 2050, 17359, 12412, 2229, 1010, 2030, 26148, 24209, 3286, 8909, 1010, 9530, 21341, 2819, 14405, 2063, 1012, 21877, 12179, 4570, 4226, 7327, 4717, 2483, 4372, 5714, 1012, 11865, 11020, 2063, 7327, 3417, 2050, 18749, 2271, 1010, 7367, 2094, 3417, 3775, 4263, 11265, 4226, 1012, 21183, 19300, 2063, 22939, 2213, 21864, 2015, 2310, 15909, 14017, 10415, 4183, 21041, 2078, 10514, 11020, 11514, 4183, 1012, 12731, 2527, 16313, 3126, 1041, 26989, 26243, 3126, 2777, 2271, 3802, 15544, 13203, 21766, 24456, 1010, 2310, 2140, 5783, 2819, 20201, 1041, 8449, 10230, 1012, 3802, 25107, 21864, 2015, 5783, 2819, 3393, 22675, 1012, 20022, 7606, 2512, 3690, 2102, 16709, 16643, 2063, 1010, 2030, 26148, 9413, 2891, 7327, 1010, 2030, 26148, 9152, 5332, 1012, 4241, 2483, 17359, 10278, 24586, 2121, 4654, 9530, 10175, 6856, 1010, 17359, 1

>`convert_ids_to_tokens`
- `convert_tokens_to_ids`의 반대되는 기능이다.

In [ ]:
tokens = bert_tokenizer.convert_ids_to_tokens(input_ids)
print(tokens)

['this', 'is', 'a', 'tutor', '##ial', 'for', 'hugging', '##face', 'token', '##izer', '##s', '.']


>`convert_tokens_to_string(tokens)`
- `tokenize`의 반대되는 기능이다.

In [ ]:
string = bert_tokenizer.convert_tokens_to_string(tokens)
print(string)

this is a tutorial for huggingface tokenizers.


>`decode`
- 위의 두 함수를 합친 함수다 `convert_tokens_to_string(convert_ids_to_tokens(token_ids))`
- token id에서 string 형태로 변환한다.

In [ ]:
bert_tokenizer.decode(input_ids2)

'[CLS] this is a tutorial for huggingface tokenizers. [SEP]'

> `skip_special_tokens=True` 파라미터로 특수 토큰을 제거할 수 있다.

In [ ]:
bert_tokenizer.decode(input_ids2, skip_special_tokens=True)

'this is a tutorial for huggingface tokenizers.'

>`batch_decode`
- 배치 내의 전체 문장들에 대해서 `decode`를 진행한다.
- 2차원 형태의 입력 (batch_size, seq_len)을 받고, 디코딩 된 문장들의 리스트를 리턴한다.

In [ ]:
print(input_ids_batch.input_ids)
print(bert_tokenizer.batch_decode(input_ids_batch.input_ids))
print(bert_tokenizer.batch_decode(input_ids_batch.input_ids, skip_special_tokens=True))

[[101, 2023, 2003, 1037, 3722, 14108, 2742, 1012, 102], [101, 2064, 2009, 21933, 3207, 3674, 11746, 2012, 1996, 2168, 2051, 1029, 102], [101, 2017, 2064, 2224, 1996, 14108, 1035, 21933, 3207, 3853, 2000, 2079, 2009, 1012, 102], [101, 2292, 1005, 1055, 4372, 16044, 2122, 11746, 2034, 1010, 2059, 4685, 14108, 1035, 21933, 3207, 1012, 102]]
['[CLS] this is a simple batch example. [SEP]', '[CLS] can it decode multiple sentences at the same time? [SEP]', '[CLS] you can use the batch _ decode function to do it. [SEP]', "[CLS] let's encode these sentences first, then perform batch _ decode. [SEP]"]
['this is a simple batch example.', 'can it decode multiple sentences at the same time?', 'you can use the batch _ decode function to do it.', "let's encode these sentences first, then perform batch _ decode."]


## 2. BERT, KLUE BERT, mBERT tokenizer 비교

> AutoTokenizer를 사용하면 어떤 tokenizer class가 필요한지 알 필요가 없다.

In [ ]:
kluebert_tokenizer = AutoTokenizer.from_pretrained("klue/bert-base")

In [ ]:
mbert_tokenizer = AutoTokenizer.from_pretrained('bert-base-multilingual-cased')

In [ ]:
print(bert_tokenizer)
print(kluebert_tokenizer)
print(mbert_tokenizer)

BertTokenizerFast(name_or_path='bert-base-uncased', vocab_size=30522, model_max_length=512, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'})
BertTokenizerFast(name_or_path='klue/bert-base', vocab_size=32000, model_max_length=512, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'})
BertTokenizerFast(name_or_path='bert-base-multilingual-cased', vocab_size=119547, model_max_length=512, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'})


In [ ]:
english_sent = "What time will you be at the birthday party?"
korean_sent = "생일파티에 몇 시쯤 올거야?"

In [ ]:
print("BERT to eng", bert_tokenizer.tokenize(english_sent))
print("BERT to kor", bert_tokenizer.tokenize(korean_sent))

BERT to eng ['what', 'time', 'will', 'you', 'be', 'at', 'the', 'birthday', 'party', '?']
BERT to kor ['ᄉ', '##ᅢ', '##ᆼ', '##ᄋ', '##ᅵ', '##ᆯ', '##ᄑ', '##ᅡ', '##ᄐ', '##ᅵ', '##ᄋ', '##ᅦ', '[UNK]', '[UNK]', '[UNK]', '?']


In [ ]:
print("KLUE BERT to eng", kluebert_tokenizer.tokenize(english_sent))
print("KLUE BERT to kor", kluebert_tokenizer.tokenize(korean_sent))

KLUE BERT to eng ['What', 'time', 'will', 'you', 'be', 'at', 'the', 'b', '##ir', '##th', '##day', 'par', '##ty', '?']
KLUE BERT to kor ['생일', '##파티', '##에', '몇', '시', '##쯤', '올', '##거', '##야', '?']


In [ ]:
print("mBERT to eng", mbert_tokenizer.tokenize(english_sent))
print("mBERT to kor", mbert_tokenizer.tokenize(korean_sent))

mBERT to eng ['What', 'time', 'will', 'you', 'be', 'at', 'the', 'birthday', 'party', '?']
mBERT to kor ['생', '##일', '##파', '##티', '##에', '몇', '시', '##쯤', '올', '##거', '##야', '?']
